##### Imports

In [ ]:
import numpy as np
import keras
from keras.utils import to_categorical
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

#### Image visualization

In [ ]:
def showImages(images, labels, offset=0):
    class_names = ['Spots', 'Eyespots']

    fig = plt.figure(figsize=(10, 10))
    for i in range(30):
        plt.subplot(6, 5, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i + offset])
        plt.xlabel(class_names[labels[i+offset]])

    fig.patch.set_facecolor('w')
    plt.show()

#### Loading data and preprocessing

In [ ]:
X_train = np.load('Xtrain_Classification1.npy')
y_train = np.load('ytrain_Classification1.npy')


print(f"Data X: {X_train.shape[0]} images of shape: {X_train.shape[1:]}")

# Split the data into training and test sets

x_train, x_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Reshaping to 30x30x3 for 30x30 RGB images

x_train = x_train.reshape(x_train.shape[0], 30, 30, 3)
x_validation = x_validation.reshape(x_validation.shape[0], 30, 30, 3)

print(f"Training set: {x_train.shape[0]} images of shape: {x_train.shape[1:]}")

print(f"Validation set: {x_validation.shape[0]} images of shape: {x_validation.shape[1:]}")

# Checking classifcation of images

showImages(x_train, y_train, 0)

# Normalizing the data
# Making sure that values are float

x_train = x_train.astype('float32')
x_validation = x_validation.astype('float32')

# Normalizing the RGB codes by dividing it to the max RGB value

x_train = x_train / 255
x_validation = x_validation / 255

# One-hot encoding

y_train = to_categorical(y_train)
y_validation = to_categorical(y_validation)

print(f"Spot label: {y_train[3]}") # Third image is a spot
print(f"Eyespot label: {y_train[0]}") # First image is an eyespot

# Image analysis

## CNN

In [ ]:
batch_size = 128
epochs = 10
num_classes = 2
input_shape = (30, 30, 3)

### Model Construction

#### Convulutional and Pooling Layers

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#### Fully Connected Layers

In [ ]:
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes, activation='softmax'))

model.summary()

#### Compile the Model

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

#### Train the Model

In [ ]:
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_validation, y_validation))

## Visualize the Results

In [ ]:
plt.figure(figsize=(10, 10))

plt.subplot(2, 1, 1)
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.xlabel('Epoch')
plt.legend(loc='best')

validation_loss, validation_accuracy = model.evaluate(x_validation, y_validation, verbose=0)

print(f"Validation loss: {validation_loss}, Validation accuracy: {validation_accuracy}")

## Evaluate the Model

In [ ]:
score = model.evaluate(x_validation, y_validation, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
predictions = model.predict(x_validation)

showImages(x_validation, predictions.argmax(axis=1), 0)